# Interpreting the predictions and coefficients of Logistic Regression

## Set up dataframe with features and labels

In [2]:
import sys
sys.path.append('/Users/q616967/Library/Python/3.10/lib/python/site-packages')

In [3]:
import os
import pandas as pd
import json
import numpy as np

from sklearn.model_selection import train_test_split

In [4]:
PROJECT_DIR = "/Users/q616967/Workspace/thesis/uni/xai-thesis/"
FEATURES_DIR = os.path.join(PROJECT_DIR, "feature_extraction/featureExtraction/output/")
RESPONSES_DIR = os.path.join(PROJECT_DIR, "responses/")

In [133]:
features_df = pd.read_csv(os.path.join(FEATURES_DIR, "merged_features.csv"))

In [6]:
responses_filename = "formatted_turbo14081857_turbo1508_eval.json"

with open(os.path.join(RESPONSES_DIR, responses_filename), "r") as f:
    responses = json.load(f)

In [17]:
idx_outcome_dict = {int(idx):res_dict['outcome'] for idx, res_dict in responses.items()}

labels_df = pd.DataFrame.from_dict(idx_outcome_dict, columns=['outcome'], orient='index')

try:
    assert len(features_df) == len(labels_df)
except AssertionError:
    print("Length mismatch between features and labels")

data_df = pd.merge(features_df, labels_df, left_index=True, right_index=True)

## Basic Pre-processing

In [18]:
train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42)

X_train = train_df.drop(columns=['outcome'])
y_train = train_df['outcome']

X_test = test_df.drop(columns=['outcome'])
y_test = test_df['outcome']

In [19]:
# standardise the values of the features (to the same range/scale)
from sklearn.preprocessing import StandardScaler

features = X_train.columns
scaler = StandardScaler()

X_train[features] = scaler.fit_transform(X_train[features])
X_test[features] = scaler.transform(X_test[features])

In [20]:
# features in X_train where all the values are the same - constant features
constant_features = X_train.columns[X_train.nunique() == 1]

# drop constant features from both X_train and X_test
X_train.drop(labels=constant_features, axis=1, inplace=True)
X_test.drop(labels=constant_features, axis=1, inplace=True)

In [55]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# working with 20 features for simplicity
selector = SelectKBest(f_classif, k=20)

X_train_kbest = selector.fit_transform(X_train, y_train)

# idx of selected features
selected_idx = selector.get_support(indices=True)
# names of selected features
selected_features = X_train.columns[selected_idx]

# df with only the selected features (full data) + outcome
data_kbest = data_df[selected_features]
data_kbest = data_kbest.join(data_df['outcome'])

# df with only the selected features (after train/test split)
X_train_kbest = X_train[selected_features]
X_test_kbest = X_test[selected_features]

## Classification and interpretation

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(random_state=1)
model.fit(X_train_kbest, y_train)

y_pred = model.predict(X_test_kbest) # returns predicted class labels
y_pred_proba = model.predict_proba(X_test_kbest) # returns predicted probabilities

accuracy = accuracy_score(y_test, y_pred)

In [63]:
y_pred[:10], y_pred_proba[:10]

(array([ True,  True,  True, False,  True,  True, False,  True, False,
         True]),
 array([[0.12459267, 0.87540733],
        [0.28715134, 0.71284866],
        [0.38211167, 0.61788833],
        [0.64147482, 0.35852518],
        [0.27549349, 0.72450651],
        [0.39031635, 0.60968365],
        [0.56317507, 0.43682493],
        [0.14812088, 0.85187912],
        [0.59290934, 0.40709066],
        [0.43958082, 0.56041918]]))

In [57]:
model_stats = {'intercept': model.intercept_, 
                'coefficients': model.coef_,
                'score': model.score(X_train_kbest, y_train),
                'accuracy': accuracy}
model_stats

{'intercept': array([0.53091152]),
 'coefficients': array([[-0.33556115, -0.17134495,  0.36438759,  0.53155209,  0.07793284,
          0.31442706, -0.1778761 , -0.43130745, -0.21452493,  0.08669528,
          0.08669528,  0.08669528,  0.08425419, -0.46575119, -0.1360285 ,
         -0.25359424, -0.42388194, -0.02054036, -0.02054036, -0.02054036]]),
 'score': 0.65,
 'accuracy': 0.5857142857142857}

In [58]:
selected_features = selected_features.tolist()
coef = model.coef_.tolist()[0]

In [59]:
coef_df = pd.DataFrame({'feature': selected_features, 'coef': coef})
coef_df = coef_df.sort_values(by='coef', ascending=False)
coef_df

,feature,coef
3,Space_GI_neg_3,0.531552
2,Socrel_GI_neg_3,0.364388
5,attention_nouns,0.314427
10,Ord_GI_adverbs,0.086695
11,Pos_GI_adverbs,0.086695
9,Numb_GI_adverbs,0.086695
12,Timespc_Lasswell_adverbs,0.084254
4,Affloss_Lasswell_neg_3,0.077933
18,Ord_GI_adverbs_neg_3,-0.020540
17,Numb_GI_adverbs_neg_3,-0.020540


#### What do these coefficients mean??

+ a positive coefficient means that the feature contributed to a positive prediction (True)
+ a negative coefficient means that the feature contributed to a negative prediction (False)
+ large abs value means that the feature is more important
+ coef_ does not include the intercept (model.intercept_)
+ the magnitude of coefficients only makes sense if the data is normalised.
+ LR coefficients are odd ratios - transform to probabilities to get something more interpretable

--

+ probabilities: 0 to 1
+ log odds: negative infinity to positive infinity
+ odds: 0 to positive infinity

--

Given two outcomes A and B, if the probability of A is 0.5 (and of B also 0.5), then the odds are 1 (odd ratio is 1:1 - both are equally probable) and the log odds are 0

The trasformations prob -> log odds and log odds -> odds are monotonic: the greater one is, the greater the other ones are as well

We transform probs to log odds because probs have a restricted range (0 to 1). It is difficult to model a variable which has restricted range. This transformation is an attempt to get around the restricted range problem. It maps probability ranging between 0 and 1 to log odds ranging from negative infinity to positive infinity. Also, the log of odds is one of the easiest to understand and interpret.  

Logistic Regression: models the logit-transformed probability as a linear relationship with the predictor variables.

--

log(p/(1-p)) = coeff

- p is the overall probability of class 1

If p = 49/200 =  .245. The odds are .245/(1-.245) = .3245 and the log of the odds (logit) is log(.3245) = -1.12546. In other words, the intercept from the model with no predictor variables is the estimated log odds of all the datapoints belonging to class 1.  We can also transform the log of the odds back to a probability: p = exp(-1.12546)/(1+exp(-1.12546)) = .245

--

https://stats.oarc.ucla.edu/other/mult-pkg/faq/general/faq-how-do-i-interpret-odds-ratios-in-logistic-regression/

#### Model confidence

In [87]:
# random sample from test set
test_sample = pd.DataFrame(X_test_kbest.iloc[np.random.randint(0, len(X_test_kbest))]).T
test_sample

,Space_GI,Affloss_Lasswell,Socrel_GI_neg_3,Space_GI_neg_3,Affloss_Lasswell_neg_3,attention_nouns,Virtue_GI_nouns,Affloss_Lasswell_nouns,Work_GI_verbs,Numb_GI_adverbs,Ord_GI_adverbs,Pos_GI_adverbs,Timespc_Lasswell_adverbs,attention_nouns_neg_3,polarity_nouns_neg_3,Causal_GI_nouns_neg_3,Affloss_Lasswell_nouns_neg_3,Numb_GI_adverbs_neg_3,Ord_GI_adverbs_neg_3,Pos_GI_adverbs_neg_3
39,0.682836,-0.136119,-0.572866,0.863774,-0.125198,-0.697388,0.189828,-0.085203,-0.449542,0.839064,0.839064,0.839064,1.101521,-0.690602,-0.734356,-0.449195,-0.085058,0.783898,0.783898,0.783898


In [94]:
one_pred_proba = model.predict_proba(test_sample)
pred_class = one_pred_proba.argmax()

print(f'predicted class = {pred_class} | confidence = {max(one_pred_proba[0])}')

predicted class = 1 | confidence = 0.782471014272346


#### Sanity check

In [120]:
'''
y_pred: predicted class label (True/False) - numpy array
y_test: gold standard class label (True/False) - pandas series
y_pred_proba: predicted probabilties for each class - numpy array

check that the max probability corresponds to the predicted class label
and that 0 = False and 1 = True
'''

assert len(y_pred) == len(y_test) == len(y_pred_proba)

for i in range(len(y_pred)):
    bool_label = y_pred[i]
    probs = y_pred_proba[i]
    num_label = probs.argmax()

    if i in [5,25,50]:
        print(f"bool_label = {bool_label} | num_label = {num_label} | probs = {probs}")
    
    if probs[0] > probs[1]:
        assert bool_label == False
        assert num_label == 0
    elif probs[0] < probs[1]:
        assert bool_label == True
        assert num_label == 1

bool_label = True | num_label = 1 | probs = [0.39031635 0.60968365]
bool_label = True | num_label = 1 | probs = [0.22359629 0.77640371]
bool_label = False | num_label = 0 | probs = [0.71415173 0.28584827]


### Transforming the coefficients

In [124]:
# transform log odds (coef) to odds and to probabilities
coef_df['odds'] = coef_df['coef'].apply(lambda x: np.exp(x))
coef_df['probs'] = coef_df['odds'].apply(lambda x: x/(1+x))
coef_df

,feature,coef,odds,probs
3,Space_GI_neg_3,0.531552,1.701571,0.629845
2,Socrel_GI_neg_3,0.364388,1.439632,0.590102
5,attention_nouns,0.314427,1.369474,0.577965
10,Ord_GI_adverbs,0.086695,1.090564,0.521660
11,Pos_GI_adverbs,0.086695,1.090564,0.521660
9,Numb_GI_adverbs,0.086695,1.090564,0.521660
12,Timespc_Lasswell_adverbs,0.084254,1.087905,0.521051
4,Affloss_Lasswell_neg_3,0.077933,1.081050,0.519473
18,Ord_GI_adverbs_neg_3,-0.020540,0.979669,0.494865
17,Numb_GI_adverbs_neg_3,-0.020540,0.979669,0.494865


Interpretations of coefficients depends on the INTERACTIONS between features